In [1]:
# Transformers installation
# ! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

```bash
pip install transformers datasets evaluate -q
pip install jiwer -q
```

# 1. Imports 

In [1]:
import PIL.Image
import torchvision
import io
import requests
import datasets
import transformers
from transformers import TrainingArguments, Trainer
import torch
from transformers import AutoProcessor

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coco = datasets.load_dataset("UCSC-VLAA/Recap-COCO-30K")['train'].train_test_split(test_size=0.3)

In [3]:
coco

DatasetDict({
    train: Dataset({
        features: ['image_id', 'coco_url', 'image', 'caption', 'recaption'],
        num_rows: 21352
    })
    test: Dataset({
        features: ['image_id', 'coco_url', 'image', 'caption', 'recaption'],
        num_rows: 9152
    })
})

In [4]:
from transformers import AutoProcessor

checkpoint = "microsoft/git-base"
processor = AutoProcessor.from_pretrained(checkpoint)

In [5]:
#processor(coco['train'][:5]['image'],coco['train'][:5]['caption'])

In [6]:
def transforms(example_batch):
    images = [x for x in example_batch["image"]]
    captions = [x for x in example_batch["caption"]]
    inputs = processor(images=images, text=captions, padding="max_length")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs


coco['train'].set_transform(transforms)
coco['test'].set_transform(transforms)

In [7]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [10]:
from evaluate import load
import torch

wer = load("wer")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predicted = logits.argmax(-1)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = processor.batch_decode(predicted, skip_special_tokens=True)
    wer_score = wer.compute(predictions=decoded_predictions, references=decoded_labels)
    return {"wer_score": wer_score}

In [9]:
from transformers import TrainingArguments, Trainer

model_name = checkpoint.split("/")[1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-pokemon",
    learning_rate=5e-5,
    num_train_epochs=50,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # gradient_accumulation_steps=2,
    # save_total_limit=3,
    # eval_strategy="steps",
    # eval_steps=50,
    # save_strategy="steps",
    # save_steps=50,
    # logging_steps=50,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    load_best_model_at_end=True,
)

ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.NO
- Save strategy: IntervalStrategy.STEPS

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=coco['train'],
    eval_dataset=coco['test'],
    compute_metrics=compute_metrics,
)

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [12]:
trainer.train()

Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.84 GiB. GPU 0 has a total capacity of 23.68 GiB of which 9.76 GiB is free. Including non-PyTorch memory, this process has 13.89 GiB memory in use. Of the allocated memory 11.84 GiB is allocated by PyTorch, and 1.73 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)